In [1]:
import numpy as np
import tenpy
from tenpy.networks.mps import MPS
from tenpy.models.hubbard import BoseHubbardChain
from tenpy.algorithms import dmrg
from tenpy.linalg import np_conserved as npc
from tenpy.models.model import CouplingMPOModel
from tenpy.tools.params import asConfig
from tenpy.networks.site import BosonSite, Site, SpinSite
from tenpy.models import lattice
from tenpy.networks.site import multi_sites_combine_charges
import os
import matplotlib.pyplot as plt

In [2]:
'''

Function to fix the external boundary links according to a given configuration.

Specify the label:   'staggered'             -> alternation of positive and negative links
                     'walls_opposite'        -> opposite "walls" of positive and negative links
                     'walls_equal'           -> equal "walls" of positive and negative links
                     'all_edges_equal'       -> same pattern (alternate + and -) for all the edges of the lattices

See the Overleaf's notes for the graphical plots of boundary configurations.

The function returns two arrays: bc_x, bc_y of size 2*L_{x,y}, i.e. the total number of boundary sites,
but only 2*(L{x,y}-2) components of these array, i.e. the number of involved boundary sites, 
are specifying the signs of the external electric links. The corners (4 components) are set to zero.

This structure is chosen to preserve & facilitate the index scrolling within the code.

The first halfs of the arrays regard the lower/leftmost external links, while the second half the
upper/rightmost external links, where we mean x/y arrays respectively.

'''

def fix_boundary_links(label,Lx,Ly):
    
    # Array storing the boundary signs
    bc_x=np.zeros(2*Lx,dtype='float'); bc_y=np.zeros(2*Ly,dtype='float')
    
    if label=='staggered':
        # Fix first half of the arrays
        # x: lower part of the lattice
        for x in range(1,Lx-1):
            if x%2==0: 
                bc_x[x]=0.5
            else:
                bc_x[x]=-0.5
        # y: leftmost part of the lattice
        for y in range(1,Ly-1):
            if y%2==0:
                bc_y[y]=-0.5
            else:
                bc_y[y]=0.5
        
        # Fix second half of the arrays
        # x: higher part of the lattice
        for x in range(Lx+1,2*Lx-1):
            if x%2==0:
                bc_x[x]=0.5
            else:
                bc_x[x]=-0.5
        # y: rightmost part of the lattice
        for y in range(Ly+1,2*Ly-1):
            if y%2==0:
                bc_y[y]=-0.5
            else: 
                bc_y[y]=0.5
        
    elif label=='walls_opposite':
        # Fix first half of the arrays
        # x: lower part of the lattice
        for x in range(1,Lx-1):
            bc_x[x]=-0.5
        # y: leftmost part of the lattice
        for y in range(1,Ly-1):
            bc_y[y]=0.5
        
        # Fix second half of the arrays
        # x: higher part of the lattice
        for x in range(Lx+1,2*Lx-1):
            bc_x[x]=0.5
        # y: rightmost part of the lattice
        for y in range(Ly+1,2*Ly-1):
            bc_y[y]=-0.5
        
        
    elif label=='walls_equal':
        # Fix first half of the arrays
        # x: lower part of the lattice
        for x in range(1,Lx-1):
            bc_x[x]=0.5
        # y: leftmost part of the lattice
        for y in range(1,Ly-1):
            bc_y[y]=-0.5
        
        # Fix second half of the arrays
        # x: higher part of the lattice
        for x in range(Lx+1,2*Lx-1):
            bc_x[x]=0.5
        # y: rightmost part of the lattice
        for y in range(Ly+1,2*Ly-1):
            bc_y[y]=-0.5
        
    elif label=='all_edges_equal':
        # Fix first half of the arrays
        # x: lower part of the lattice
        for x in range(1,Lx-1):
            if x%2==0: 
                bc_x[x]=-0.5
            else:
                bc_x[x]=0.5
        # y: leftmost part of the lattice
        for y in range(1,Ly-1):
            if y%2==0:
                bc_y[y]=0.5
            else:
                bc_y[y]=-0.5
        
        # Fix second half of the arrays
        # x: higher part of the lattice
        for x in range(Lx+1,2*Lx-1):
            if x%2==0:
                bc_x[x]=0.5
            else:
                bc_x[x]=-0.5
        # y: rightmost part of the lattice
        for y in range(Ly+1,2*Ly-1):
            if y%2==0:
                bc_y[y]=-0.5
            else: 
                bc_y[y]=0.5
        
    else:
        print("!!! label does not correspond to any allowed boundary configuration !!!")
              
    return bc_x, bc_y

In [3]:
'''

Useful functions to extract observables: 

- total flippability
- sublattice flippabilities
- total "susceptibility" (check definition)
- checkerboard pattern

'''

#for i in range(len(initial_MPS)):
#    print(i,M.lat.mps2lat_idx(i))

# Total flippability of the lattice
# Return the number of plaquette just as a check, it can be removed

def total_flippability(psi,M,Lx,Ly):
    
    total_flip=0.0
    
    # Flippability for all the plaquettes
    for x in range(Lx-1):
        for y in range(Ly-1):
            #site_parity=(-1)**(x+y)
            plaq_flipp=psi.expectation_value_term([('Pplus', M.lat.lat2mps_idx([x,y,0])),('Pplus', M.lat.lat2mps_idx([x+1,y,1])),('Pminus', M.lat.lat2mps_idx([x,y+1,0])),('Pminus', M.lat.lat2mps_idx([x,y,1]))])
            # Add to total flippability weighted with parity
            # total_flip+=site_parity*plaq_flipp
            total_flip+=plaq_flipp
    
    return total_flip

# Total susceptibility of the lattice
# Return the sum of the squares of flippabilities computed in each sublattice: Ma**2+Mb**2=chi

def total_susceptibility(psi,M,Lx,Ly):
    
    Ma=0.0; Mb=0.0
    
    # Define total flippability in each sublattice
    for x in range(Lx-1):
        for y in range(Ly-1):
            site_parity=(-1)**(x+y)
            if site_parity==1:
                # A sublattice
                Ma+=psi.expectation_value_term([('Pplus', M.lat.lat2mps_idx([x,y,0])),('Pplus', M.lat.lat2mps_idx([x+1,y,1])),('Pminus', M.lat.lat2mps_idx([x,y+1,0])),('Pminus', M.lat.lat2mps_idx([x,y,1]))])
            else:
                Mb+=psi.expectation_value_term([('Pplus', M.lat.lat2mps_idx([x,y,0])),('Pplus', M.lat.lat2mps_idx([x+1,y,1])),('Pminus', M.lat.lat2mps_idx([x,y+1,0])),('Pminus', M.lat.lat2mps_idx([x,y,1]))])
    
    # Define total susceptibility
    total_chi=Ma**2+Mb**2
    return Ma, Mb, total_chi
    
# Test function: returns the checkerboard pattern on our lattice
def checkerboard(Lx,Ly):
    
    sublat_A=[]; sublat_B=[]
    
    for x in range(Lx-1):
        for y in range(Ly-1):
            parity=(-1)**(x+y)
            if parity==1:
                sublat_A.append([x,y])
            else:
                sublat_B.append([x,y])
    
    return sublat_A,sublat_B

In [9]:
class plaquette_model(CouplingMPOModel):
    
    def init_lattice(self, model_params): 
        
        # Gauge field 
        S = model_params.get('S', 2) # Cutoff gauge field default is 5 (spin 2)
        # Matter field
        n_max = model_params.get('n_max', 1) # Matter Hilbert space dimension - hardcore boson default
        filling = model_params.get('filling', 0.5) # Default filling

        # Define gauge field as spin-S site
        gauge_field = SpinSite(S=S, conserve=None)
        
        # Add new operators for the gauge field site
        d = 2 * S + 1
        d = int(d)
        Sz_diag = -S + np.arange(d)
        
        Sz = np.diag(Sz_diag)
        sigmam = np.zeros([d, d])
        for n in np.arange(d - 1):
            sigmam[n + 1, n] = 1
        sigmap = np.transpose(sigmam)
        SzSz = Sz@Sz
        
        gauge_field.add_op('sigmap', sigmap)
        gauge_field.add_op('sigmam', sigmam)
        gauge_field.add_op('SzSz', SzSz)
        gauge_field.add_op('sigmaz', Sz)
        gauge_field.add_op('Pplus', Sz+np.identity(2)/2)
        gauge_field.add_op('Pminus', Sz-np.identity(2)/2)
        
        # Add matter fields as boson sites (uncomment when matter is added)
        
        #conserve_matter_field = model_params.get('conserve_boson', 'N') 
        #matter_field = BosonSite(Nmax=n_max, conserve=conserve_matter_field, filling=filling)
        #multi_sites_combine_charges([matter_field, gauge_field]) # combine charges so that everything is consistent if we add matter fields
        
        Lx = model_params.get('Lx', 0.) # Lattice size x
        Ly = model_params.get('Ly', 0.) # Lattice size y
        bc_MPS = model_params.get('bc_MPS', 'finite') # Boundary conditions - finite or infinite

        # Define lattice (only gauge field): order in unit cell is x,y
        lat = lattice.Lattice([Lx, Ly], [gauge_field, gauge_field], bc_MPS=bc_MPS) 
        
        # Define lattice with matter field: order is x, y and matter field onsite
        #lat = lattice.Lattice([Lx, Ly], [gauge_field, gauge_field, matter_field], bc_MPS=bc_MPS) 
        return lat

    def init_terms(self, model_params):
        
        # Get interaction strengths:
        # g: kinetic coupling (J=1/2/g**2)
        # t: hopping (with matter)
        # lam_penalty: penalty term coupling
        # lambda_RK: RK ccoefficient
        # bc_label: label specifying the boundary conditions
        
        g = model_params.get('g', 1.); J = 1/2/abs(g)**2
        t = model_params.get('t', 1.)
        lam_penalty = model_params.get('lam_penalty',1.)
        lambda_RK = model_params.get('lam_RK',1.)
        bc_label = model_params.get('bc_gaugefield','staggered')

        # ADD ELECTRIC FIELD TO HAMILTONIAN (SzSz interaction, constant for S=1/2)
        self.add_onsite(abs(g)**2/2, 0, 'SzSz') # Gauge field direction x
        self.add_onsite(abs(g)**2/2, 1, 'SzSz') # Gauge field direction y
        
        # ADD PLAQUETTE KINETIC TERMS
        for y in range(Ly-1):
            for x in range(Lx-1):          
                self.add_local_term(-J, [('sigmap', [x,y,0]),('sigmap', [x+1,y,1]),('sigmam', [x,y+1,0]),('sigmam', [x,y,1])])
                self.add_local_term(-J, [('sigmap', [x,y,1]),('sigmap', [x,y+1,0]),('sigmam', [x+1,y,1]),('sigmam', [x,y,0])]) # + h.c.
        
        # ADD RK TERMS
        for y in range(Ly-1):
            for x in range(Lx-1):          
                self.add_local_term(lambda_RK, [('Pplus', [x,y,0]),('Pplus', [x+1,y,1]),('Pminus', [x,y+1,0]),('Pminus', [x,y,1])])
                self.add_local_term(lambda_RK, [('Pplus', [x,y,1]),('Pplus', [x,y+1,0]),('Pminus', [x+1,y,1]),('Pminus', [x,y,0])]) # + h.c.
               
        # ADD ENERGY PENALTY TERMS (brute force Gauss law imposition)
        
        '''
        Structure of the Gauss law at vertex (signs):
        
                |
              2 |
                |
        1  ----------- 3
                |
             4  |
                |
                
        G = E1+E4-E2-E3
        
        !! SPIN 1/2: set q_link different from zero if finite BC are considered. !!
        
        '''
        
        # Set the fixed boundary link parameters 
        # Allowed boundary configs: 'staggered', 'walls_opposite', 'walls_equal'
        values_bc_x,values_bc_y=fix_boundary_links(bc_label,Lx,Ly)
        
        # Bulk terms
        for x in range(1,Lx-1):
            for y in range(1,Ly-1):
                self.add_local_term(lam_penalty,[('sigmaz',[x,y,0]),('sigmaz',[x,y,0])])
                self.add_local_term(lam_penalty,[('sigmaz',[x,y,1]),('sigmaz',[x,y,1])])
                self.add_local_term(lam_penalty,[('sigmaz',[x-1,y,0]),('sigmaz',[x-1,y,0])])
                self.add_local_term(lam_penalty,[('sigmaz',[x,y-1,1]),('sigmaz',[x,y-1,1])])
                self.add_local_term(2*lam_penalty,[('sigmaz',[x,y,0]),('sigmaz',[x,y,1])])
                self.add_local_term(-2*lam_penalty,[('sigmaz',[x,y,1]),('sigmaz',[x-1,y,0])])
                self.add_local_term(2*lam_penalty,[('sigmaz',[x-1,y,0]),('sigmaz',[x,y-1,1])])
                self.add_local_term(-2*lam_penalty,[('sigmaz',[x,y-1,1]),('sigmaz',[x,y,0])])
                self.add_local_term(-2*lam_penalty,[('sigmaz',[x,y,0]),('sigmaz',[x-1,y,0])])
                self.add_local_term(-2*lam_penalty,[('sigmaz',[x,y,1]),('sigmaz',[x,y-1,1])])
            
        # Vertex terms
        # (0,0)
        self.add_local_term(lam_penalty,[('sigmaz',[0,0,0]),('sigmaz',[0,0,0])])
        self.add_local_term(lam_penalty,[('sigmaz',[0,0,1]),('sigmaz',[0,0,1])])
        self.add_local_term(2*lam_penalty,[('sigmaz',[0,0,0]),('sigmaz',[0,0,1])])
        
        # (0,L)
        self.add_local_term(lam_penalty,[('sigmaz',[0,Ly-1,0]),('sigmaz',[0,Ly-1,0])])
        self.add_local_term(lam_penalty,[('sigmaz',[0,Ly-2,1]),('sigmaz',[0,Ly-2,1])])
        self.add_local_term(-2*lam_penalty,[('sigmaz',[0,Ly-1,0]),('sigmaz',[0,Ly-2,1])])
        
        # (L,0)
        self.add_local_term(lam_penalty,[('sigmaz',[Lx-2,0,0]),('sigmaz',[Lx-2,0,0])])
        self.add_local_term(lam_penalty,[('sigmaz',[Lx-1,0,1]),('sigmaz',[Lx-1,0,1])])
        self.add_local_term(-2*lam_penalty,[('sigmaz',[Lx-2,0,0]),('sigmaz',[Lx-1,0,1])])
        
        # (L,L)
        self.add_local_term(lam_penalty,[('sigmaz',[Lx-2,Ly-1,0]),('sigmaz',[Lx-2,Ly-1,0])])
        self.add_local_term(lam_penalty,[('sigmaz',[Lx-1,Ly-2,1]),('sigmaz',[Lx-1,Ly-2,1])])
        self.add_local_term(2*lam_penalty,[('sigmaz',[Lx-2,Ly-1,0]),('sigmaz',[Lx-1,Ly-2,1])])
        
        
        # Horizontal boundaries
               
        for x in range(1,Lx-1):
            # Lower boundary
            self.add_local_term(lam_penalty,[('sigmaz',[x,0,0]),('sigmaz',[x,0,0])])
            self.add_local_term(lam_penalty,[('sigmaz',[x,0,1]),('sigmaz',[x,0,1])])
            self.add_local_term(lam_penalty,[('sigmaz',[x-1,0,0]),('sigmaz',[x-1,0,0])])
            self.add_local_term(2*lam_penalty,[('sigmaz',[x,0,0]),('sigmaz',[x,0,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[x-1,0,0]),('sigmaz',[x,0,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[x-1,0,0]),('sigmaz',[x,0,0])])
            #Fictitious external link contributions (Low)
            self.add_local_term(2*lam_penalty*values_bc_x[x],[('sigmaz',[x-1,0,0])])
            self.add_local_term(-2*lam_penalty*values_bc_x[x],[('sigmaz',[x,0,1])])
            self.add_local_term(-2*lam_penalty*values_bc_x[x],[('sigmaz',[x,0,0])])
        
            # Upper boundary            
            self.add_local_term(lam_penalty,[('sigmaz',[x,Ly-1,0]),('sigmaz',[x,Ly-1,0])])
            self.add_local_term(lam_penalty,[('sigmaz',[x,Ly-2,1]),('sigmaz',[x,Ly-2,1])])
            self.add_local_term(lam_penalty,[('sigmaz',[x-1,Ly-1,0]),('sigmaz',[x-1,Ly-1,0])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[x,Ly-1,0]),('sigmaz',[x,Ly-2,1])])
            self.add_local_term(2*lam_penalty,[('sigmaz',[x-1,Ly-1,0]),('sigmaz',[x,Ly-2,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[x-1,Ly-1,0]),('sigmaz',[x,Ly-1,0])])
            #Fictitious external link contributions (Up)
            self.add_local_term(-2*lam_penalty*values_bc_x[Lx+x],[('sigmaz',[x-1,Ly-1,0])])
            self.add_local_term(-2*lam_penalty*values_bc_x[Lx+x],[('sigmaz',[x,Ly-2,1])])
            self.add_local_term(2*lam_penalty*values_bc_x[Lx+x],[('sigmaz',[x,Ly-1,0])])
            
            
        # Vertical boundaries
        for y in range(1,Ly-1):
            # Left-most boundary
            self.add_local_term(lam_penalty,[('sigmaz',[0,y,0]),('sigmaz',[0,y,0])])
            self.add_local_term(lam_penalty,[('sigmaz',[0,y,1]),('sigmaz',[0,y,1])])
            self.add_local_term(lam_penalty,[('sigmaz',[0,y-1,1]),('sigmaz',[0,y-1,1])])
            self.add_local_term(2*lam_penalty,[('sigmaz',[0,y,0]),('sigmaz',[0,y,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[0,y,0]),('sigmaz',[0,y-1,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[0,y,1]),('sigmaz',[0,y-1,1])])
            #Fictitious external link contributions (Left)
            self.add_local_term(2*lam_penalty*values_bc_y[y],[('sigmaz',[0,y-1,1])])
            self.add_local_term(-2*lam_penalty*values_bc_y[y],[('sigmaz',[0,y,1])])
            self.add_local_term(-2*lam_penalty*values_bc_y[y],[('sigmaz',[0,y,0])])
        
            # Right-most boundary
            self.add_local_term(lam_penalty,[('sigmaz',[Lx-2,y,0]),('sigmaz',[Lx-2,y,0])])
            self.add_local_term(lam_penalty,[('sigmaz',[Lx-1,y,1]),('sigmaz',[Lx-1,y,1])])
            self.add_local_term(lam_penalty,[('sigmaz',[Lx-1,y-1,1]),('sigmaz',[Lx-1,y-1,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[Lx-2,y,0]),('sigmaz',[Lx-1,y,1])])
            self.add_local_term(2*lam_penalty,[('sigmaz',[Lx-2,y,0]),('sigmaz',[Lx-1,y-1,1])])
            self.add_local_term(-2*lam_penalty,[('sigmaz',[Lx-1,y,1]),('sigmaz',[Lx-1,y-1,1])])
            #Fictitious external link contributions (Right)
            self.add_local_term(-2*lam_penalty*values_bc_y[Ly+y],[('sigmaz',[Lx-2,y,0])])
            self.add_local_term(-2*lam_penalty*values_bc_y[Ly+y],[('sigmaz',[Lx-1,y-1,1])])
            self.add_local_term(2*lam_penalty*values_bc_y[Ly+y],[('sigmaz',[Lx-1,y,1])])
    

        # ADD MEDIATED HOPPING (when we add matter fields)
        # Direction x 
        # (Note the third index is the unit cell index, x corresponds to position 0 and y position 1 - matter position 2). 
        #for y in range(Ly-1):
        #    for x in range(Lx-1):  
        #        self.add_local_term(t, [('Bd', [x,y,2]),('sigmap',[x,y,0]),('B', [x+1,y,2])])
        #        self.add_local_term(t, [('B', [x,y,2]),('sigmap',[x,y,0]),('Bd', [x+1,y,2])]) # +h.c.
                
        # Direction y
        #for x in range(Lx):
        #    for y in range(Ly-1):
        #        self.add_local_term(t, [('Bd', [x,y,2]),('sigmap',[x,y,1]),('B', [x,y+1,2])])
        #        self.add_local_term(t, [('B', [x,y,2]),('sigmap',[x,y,1]),('Bd', [x,y+1,2])]) # +h.c.
         
        # IN CASE WE WANNA ADD PERTURBATIONS (maybe later to fix the corner states?)
        #self.add_local_term(100, [('Sz', [Lx-1,Ly-1,2])])
        #self.add_local_term(100, [('Sz', [0,0,2])])

In [10]:
class lgt_hoti(plaquette_model, CouplingMPOModel):

    def __init__(self, model_params):
        
        model_params = asConfig(model_params, self.__class__.__name__)
        model_params.setdefault('lattice', 'Square')
        CouplingMPOModel.__init__(self, model_params)

In [11]:
def DMRG(params, Lx, Ly, initial_vector, filling=0.5, S = 2, n_max = 1, chi_max = 30, bc_MPS = 'finite', mixer=True, conserve='N', orthogonal=[]):
       
    #Run DMRG algorithm 
    
    model_params = dict(S=S, n_max=n_max, filling=filling, bc_MPS=bc_MPS, Lx=Lx, Ly=Ly, t=params['t'], g=params['g'], lam_penalty=params['lam_penalty'], lam_RK=params['lam_RK'],bc_gaugefield=params['bc_gaugefield'], conserve_boson=conserve, conserve_spin=None, verbose=0)
    M = lgt_hoti(model_params)
    # Construct MPS from initial state (vector as input)
    psi = MPS.from_product_state(M.lat.mps_sites(), initial_vector, bc=M.lat.bc_MPS)    
               
    dmrg_params = {                                                                                             
        'mixer': mixer,                                                                                          
        'trunc_params': {                                                                                       
        'chi_max': chi_max,                                                                                                                                                                    
        },                                                                                                      
        'max_E_err': 1.e-16,                                                                                    
        'verbose': 1,
        'orthogonal_to': orthogonal}
    
    info = dmrg.run(psi, M, dmrg_params)
    
    return info, psi

In [13]:
#############################################################
# Single DMRG run
#############################################################

# PARAMS
Lx = 4
Ly = 4
bc_label='all_edges_equal'
params = dict(t=0, g=np.sqrt(0.5), lam_penalty=40.0, lam_RK=-2.5, bc_gaugefield=bc_label)    # g^2=0.5; g^2=1.5
filling = 0.5
chi_max = 50
n_max = 1
S = 0.5
bc_MPS = 'finite'
conserve='N'

M = lgt_hoti(dict(S=S, n_max=n_max, filling=filling, bc_MPS=bc_MPS, Lx=Lx, Ly=Ly, t=params['t'], g=params['g'], lam_penalty=params['lam_penalty'], lam_RK=params['lam_RK'],bc_gaugefield=params['bc_gaugefield'], conserve_boson=conserve, conserve_spin=None, verbose=0))

# Construct snake vector for initial MPS 
# This will change when adding matter fields to Lx*Ly*3.

len_initialMPS = 2*Lx*Ly

initial_MPS = np.zeros(len_initialMPS,dtype='int')

if S==0.5:
    initial_MPS=np.tile([0,1],int(len_initialMPS/2))
    # Random shuffled initial MPS
    np.random.shuffle(initial_MPS)
else:
    initial_MPS=np.random.randint(2*S+1, size=len_initialMPS)

print(initial_MPS)

info, psi = DMRG(params, Lx, Ly, initial_MPS, filling=filling, S=S, n_max = 1, chi_max=chi_max, bc_MPS = 'finite')
np.save('Z2'+bc_label+'_50_psi_g_%.2f_t_%.2f_penalty_%.2f_RKterm_%.2f_L_%.0f_S_%.1f.npy' %(params['g'], params['t'], params['lam_penalty'], params['lam_RK'], Lx*Ly, S), [psi])
#print(info)

C:\Users\pieri\anaconda3\envs\tenpy\Lib\site-packages\tenpy\tools\params.py:237: UserWarning: unused options for config lgt_hoti:
['conserve_boson', 'conserve_spin', 'verbose']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
C:\Users\pieri\anaconda3\envs\tenpy\Lib\site-packages\tenpy\tools\params.py:237: UserWarning: unused options for config lgt_hoti:
['conserve_boson', 'conserve_spin', 'verbose']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
C:\Users\pieri\anaconda3\envs\tenpy\Lib\site-packages\tenpy\algorithms\mps_common.py:272: FutureWarning: Deprecated `orthogonal_to` in dmrg options: instead give `orthogonal_to` as keyword to the Algorithm class.
  warnings.warn(


[1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1]


final DMRG state not in canonical form up to norm_tol=1.00e-05: norm_err=8.05e-01
C:\Users\pieri\anaconda3\envs\tenpy\Lib\site-packages\tenpy\tools\params.py:237: UserWarning: unused option ['verbose'] for config DMRG

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [ ]:
# Compute observables
Oflip=total_flippability(psi,M,Lx,Ly)
Ma,Mb,chi=total_susceptibility(psi,M,Lx,Ly)
print(Oflip/Lx/Ly)
print(chi/Lx/Ly)

In [ ]:
#############################################################
# Cycle over parameters (multiple DMRG runs)
#############################################################

# PURPOSE: REPRODUCE FIG. 6 OF PAOLO'S PRD

# Directory for the files
main_directory="DATA_PSI_100chi_151123"
if not os.path.exists(main_directory):
    os.makedirs(main_directory)
    
# PARAMS
Lx = 5
Ly = 5
filling = 0.5
n_max = 1
S = 0.5
bc_MPS = 'finite'
conserve='N'

chi_max_values=[100]
lambda_values=np.linspace(-3.0,3.0,num=30)
Oflip_values=[]; chi_values=[]; Ma_list=[]; Mb_list=[]

for chi_value in chi_max_values:
    
    for lamRK in lambda_values:
        
        chi_max = chi_value
        
        print("--------------------------------------")
        print("lambda_RK={}".format(lamRK))
        print("--------------------------------------")

        params = dict(t=0, g=np.sqrt(0.5), lam_penalty=40.0, lam_RK=lamRK, bc_gaugefield='staggered')    # g^2=0.5 means J=1 in the RK model (see our notes)

        # Perform the DMRG routine
        M = lgt_hoti(dict(S=S, n_max=n_max, filling=filling, bc_MPS=bc_MPS, Lx=Lx, Ly=Ly, t=params['t'], g=params['g'], lam_penalty=params['lam_penalty'], lam_RK=params['lam_RK'], bc_gaugefield=params['bc_gaugefield'], conserve_boson=conserve, conserve_spin=None, verbose=0))

        # Construct snake vector for initial MPS 
        # This will change when adding matter fields to Lx*Ly*3.

        len_initialMPS = 2*Lx*Ly

        initial_MPS = np.zeros(len_initialMPS,dtype='int')

        if S==0.5:
            initial_MPS=np.tile([0,1],int(len_initialMPS/2))
            # Random shuffled initial MPS
            np.random.shuffle(initial_MPS)
        else:
            initial_MPS=np.random.randint(2*S+1, size=len_initialMPS)

        info, psi = DMRG(params, Lx, Ly, initial_MPS, filling=filling, S=S, n_max = 1, chi_max=chi_max, bc_MPS = 'finite')
        psi_filename=main_directory+'/psi_g_%.2f_t_%.2f_penalty_%.2f_RKterm_%.2f_L_%.0f_S_%.1f.npy' %(params['g'], params['t'], params['lam_penalty'], params['lam_RK'], Lx*Ly, S)
        np.save(psi_filename, [psi])

In [ ]:
# Plots and observables
L_list=[3,5]
# Directory for the files
main_directory="DATA_PSI_50chi_141123"
if not os.path.exists(main_directory):
	os.makedirs(main_directory)
    
lambda_values=np.linspace(-3.0,3.0,num=30)
filling = 0.5
n_max = 1
S = 0.5
bc_MPS = 'finite'
conserve='N'

Oflip_values=[]; chi_values=[]
Ma_list=[]; Mb_list=[]

print(len(lambda_values))

for L in L_list:
    
    Lx=L; Ly=L
    
    temp_Oflip=[]; temp_chi=[]; temp_Ma=[]; temp_Mb=[]
    
    for lamRK in lambda_values:
        
        params = dict(t=0, g=np.sqrt(0.5), lam_penalty=40.0, lam_RK=lamRK)    # g^2=0.5 means J=1 in the RK model (see our notes)

        # Perform the DMRG routine
        M = lgt_hoti(dict(S=S, n_max=n_max, filling=filling, bc_MPS=bc_MPS, Lx=Lx, Ly=Ly, t=params['t'], g=params['g'], lam_penalty=params['lam_penalty'], lam_RK=params['lam_RK'], conserve_boson=conserve, conserve_spin=None, verbose=0))

        psi_filename=main_directory+'/psi_g_%.2f_t_%.2f_penalty_%.2f_RKterm_%.2f_L_%.0f_S_%.1f.npy' %(params['g'], params['t'], params['lam_penalty'], params['lam_RK'], Lx*Ly, S)
    
        psi = np.load(psi_filename, allow_pickle=True)[0]
    
        # Compute total flippability (whole lattice)

        Oflip=total_flippability(psi,M,Lx,Ly)
        
        # Compute susceptibility (whole lattice)
        Ma,Mb,chi=total_susceptibility(psi,M,Lx,Ly)
        
        temp_Oflip.append(Oflip); temp_chi.append(chi); temp_Ma.append(Ma); temp_Mb.append(Mb)
    
    Oflip_values.append(temp_Oflip)
    chi_values.append(temp_chi)
    Ma_list.append(temp_Ma); Mb_list.append(temp_Mb)

In [ ]:
# Flippability plot

plt.rcParams['text.usetex'] = True
plt.figure(1)
plt.ylabel(r'$\frac{M(\lambda)}{N_{plaq}}$',fontsize=20);plt.xlabel(r'$\lambda$',fontsize=20)
for L in L_list:
    subA,subB=checkerboard(L,L)
    total_plaq_L=len(subA)+len(subB)
    plt.scatter(lambda_values,np.asarray(Oflip_values[L_list.index(L)])/total_plaq_L,label=r'$L_x=L_y={}$'.format(L))
plt.legend(loc='best')
#plt.xlim(-1.0,1.0)
plt.tight_layout()
#plt.savefig('[with L=4]Oflip_vs_lambda_{}S_{}penalty.pdf'.format(S,params['lam_penalty']),bbox_inches='tight')

In [ ]:
# Scusceptibility plot

plt.rcParams['text.usetex'] = True
plt.figure(1)
plt.ylabel(r'$\frac{\chi(\lambda)}{N_{plaq}}$',fontsize=20);plt.xlabel(r'$\lambda$',fontsize=20)
for L in L_list:
    subA,subB=checkerboard(L,L)
    total_plaq_L=len(subA)+len(subB)
    plt.scatter(lambda_values,np.asarray(chi_values[L_list.index(L)])/total_plaq_L,label=r'$L_x=L_y={}$'.format(L))
plt.legend(loc='best')
#plt.xlim(-1.0,1.0)
plt.tight_layout()
#plt.savefig('[with L=4]chi_vs_lambda_{}S_{}penalty.pdf'.format(S,params['lam_penalty']),bbox_inches='tight')

In [ ]:
# Plot of the sublattice susceptibilities for different L
# 1st set of subplots: MA and MB separately for each L

total_subplots=len(L_list)

fig, axs = plt.subplots(1, total_subplots, layout='tight')

# Normalize with the number of lattice sites belonging to each sublattice

# 1 subplot: L=3

for index_plot in range(len(L_list)):
    L=L_list[index_plot]
    subA,subB=checkerboard(L,L)
    total_plaq_L=len(subA)+len(subB)
    axs[index_plot].set_title(r'$L_x=L_y={}$'.format(L),fontsize=18)
    axs[index_plot].scatter(lambda_values,np.asarray(Ma_list[index_plot])/len(subA),label=r'$A$ sublattice')
    axs[index_plot].scatter(lambda_values,np.asarray(Mb_list[index_plot])/len(subB),label=r'$B$ sublattice')
    axs[index_plot].legend(loc='best')
    axs[index_plot].set_xlabel(r'$\lambda$',fontsize=18); axs[index_plot].set_ylabel(r'$\frac{M_{A,B}}{(N_{A,B})_{plaq}}$',fontsize=18)
    axs[index_plot].legend(loc='best')
    
plt.tight_layout()
plt.show()

#plt.savefig('[with L=4]MA_MB_vs_lambda_{}S_{}penalty.pdf'.format(S,params['lam_penalty']),bbox_inches='tight')

In [ ]:
# Plot of the sublattice susceptibilities for different L
# 2nd set of subplots: MA and MB for each L 

total_subplots=len(L_list)

fig, axs = plt.subplots(1, total_subplots, layout='tight')

# Normalize with the number of lattice sites belonging to each sublattice

for L in L_list:
    subA,subB=checkerboard(L,L)
    total_plaq_L=len(subA)+len(subB)
    axs[0].scatter(lambda_values,np.asarray(Ma_list[L_list.index(L)])/len(subA),label=r'$L_x=L_y={}$'.format(L))
    axs[0].legend(loc='best')
    axs[0].set_xlabel(r'$\lambda$',fontsize=18); axs[0].set_ylabel(r'$\frac{M_{A}}{(N_{A})_{plaq}}$',fontsize=18)
    axs[0].legend(loc='best')
        
    axs[1].scatter(lambda_values,np.asarray(Mb_list[L_list.index(L)])/len(subB),label=r'$L_x=L_y={}$'.format(L))
    axs[1].legend(loc='best')
    axs[1].set_xlabel(r'$\lambda$',fontsize=18); axs[1].set_ylabel(r'$\frac{M_{B}}{(N_{B})_{plaq}}$',fontsize=18)
    axs[1].legend(loc='best')
    axs[1].yaxis.set_label_position("right")
    
plt.tight_layout()
plt.show()
#plt.savefig('[with L=4]1_MA_MB_vs_lambda_{}S_{}penalty.pdf'.format(S,params['lam_penalty']),bbox_inches='tight')